# Set Up

Ensure to have PIP installed. On Ubuntu:

```sh
# apt install python3-pip
# apt install python3-venv
```

Create your virtual environment

```sh
$ python3 -m venv ./venv
```

Then once it's created, you need to initialise it before setting the Jupyter Kernel:

```sh
$ source
 ./venv/bin/activate
```

Choose the venv Python kernel from within VSCode. Follow instructions for installing the ipykernel which is needed to run Jupiter. It's installed safely in your virtual environment.

If you can't see the venv as the kernel in VSCode, try changing to the directory where the venv folder is (not the venv folder itself, but one before it) and starting VSCode from there. Else you may unintentionally install all the packages needed in your global or user configuration.

# Simulation

This file covers the basics of the window shaping algorithm for sending data. It
allows simulating the duration it takes to send packets, to test the algorithm
independent of the OS, in a deterministic way. Then the algorithm can be
optimised and translated into C++.

## Configuration parameters

In [11]:
n = 1
m = 10
p = 107

## Simulation Parameters

The `sim_dur` is a way to simulate times that it takes to send, to test for
unexpected delays that might mimic real situations.

The `sleep_time` simulates the passage of time.

The `sleep_time_dur` is used when sending traffic, it reads the time to sleep
from `sim_dur`. If we run out of elements to sleep, we assume the simulation is
finished and raise an exception to exit.

In [12]:
import sys

sim_time = 0
sim_dur_i = 0
#sim_dur = [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ]
#sim_dur = [ 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10 ]
#sim_dur = [ 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14 ]
#sim_dur = [ 2, 2, 14, 14, 14, 14, 14, 2, 2, 2, 2, 40, 2, 2, 2, 2, 2, 2, 2, 2 ]
sim_dur = [ 2, 2, 14, 40, 14, 14, 14, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 ]
#sim_dur = [ 2, 40, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 ]
#sim_dur = [ 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7 ]
#sim_dur = [ 7, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7 ]
#sim_dur = [ 7, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7 ]

class StopExecution(Exception):
    def _render_traceback_(self):
        return []

def reset():
    global sim_time
    global sim_dur_i
    sim_time = 0
    sim_dur_i = 0

def sim_running():
    if sim_dur_i < len(sim_dur):
        return True
    return False

def sleep_time(time):
    global sim_time
    sim_time += time

def sleep_time_dur():
    global sim_dur_i
    if sim_dur_i < len(sim_dur):
        sleep_time(sim_dur[sim_dur_i])
    else:
        raise StopExecution
    sim_dur_i += 1

def get_time():
    return sim_time

def send_packets(count):
    sleep_time_dur() # Simulate sending in this time
    return count

## Implementation of the algorithm

The algorithm is a credit based sender over the time period of `m * n`
milliseconds. To tries to keep the amount of packets sent in this window
constant.

When the amount of packets sents is now out of the window, we have this many
credits for sending new packets.

### First (working) implementation

This is the first working, non-optimised implementation that can be used as a
reference.

In [13]:
sp = -1
s = 0
packet_sent_window_count = 0

window = [0] * n

def pwindow():
    f = "["
    for v in window:
        f += " {:3}".format(v)
    return f + "]"

reset()
print(f"n={n} m={m} p={p}")

total_sent = 0
while sim_running():
    if (s < n):
        packet_sent_expected = int(p * (s + 1) / n)
    else:
        packet_sent_expected = p

    sending = True
    while sim_running() and sending:
        dbg_time = get_time()
        packet_sent_remaining = packet_sent_expected - packet_sent_window_count
        print(f"t={dbg_time:4d} s={s:3} sp={sp:3} win={pwindow()} pse={packet_sent_expected:4} pswc={packet_sent_window_count:4} psr={packet_sent_remaining:4}", end=" ")

        # We shouldn't send more than this group at a time (especially if we're
        # sending multiple packets with one system call).
        packet_send_max = int(p * (s + 1) / n) - int(p * s / n)
        if packet_sent_remaining > packet_send_max:
            packet_send = packet_send_max
        else:
            packet_send = packet_sent_remaining
        sent = send_packets(packet_send)

        total_sent += sent
        time = get_time()
        print(f"sent={sent:4} t'={time:4d}", end=" ")

        if (s < n):
            packet_sent_window_count += sent
            window[s] += sent
        else:
            if (sp == s):
                # Adding to the same slot
                packet_sent_window_count += sent
                window[s % n] += sent
            else:
                # New slot
                packet_sent_window_count += sent
                window[s % n] = sent

        sf = int(time / m)
        print(f"pswc'={packet_sent_window_count:5} win'={pwindow()} sf={sf:3}", end=" ")

        sp = s
        if (sf <= s):
            if (packet_sent_expected > packet_sent_window_count):
                # If we're resending, we will add to the current slot again,
                # which means we're catching up. To smoothen out the credit
                # window, we shift back every slot if there was any window we
                # missed prior.

                # Find the next previous slot that is empty, and shift
                # everything down one slot to that one.
                #
                # We add 'n' to the index 's', so that we don't underflow.
                empty_slot = -1
                for x in range(1, n):
                    if window[(s + (n - x)) % n] == 0:
                        empty_slot = x
                        break
                if empty_slot != -1:
                    print(f"{empty_slot} {(s + (n - empty_slot)) % n}", end=" ")
                    for x in range(empty_slot, 0, -1):
                        window[(s + (n - x)) % n] = window[(s + (n - x + 1)) % n]
                        window[(s + (n - x + 1)) % n] = 0

                print("Resend!")
                continue

            s += 1

            # Increase credits for the upcoming slot.
            print(f"erase1=win[{s}=>{s%n}]={window[s%n]}", end=" ")
            packet_sent_window_count -= window[s % n]
            window[s % n] = 0

            # Given the current time and the next slot, calculate the number of
            # milliseconds to sleep
            W = s * m - time
        else:
            s = sf
            W = 0

            # Increase credits for the upcoming slot.
            for x in range(sp+1,sf+1):
                print(f"erase2=win[{x}=>{x%n}]={window[x%n]}", end=" ")
                packet_sent_window_count -= window[x % n]
                window[x % n] = 0

        sending = False
        if (W >= 4):
            print(F"Sleep({W})", end=" ")
            sleep_time(W)
        else:
            print(F"W={W}", end=" ")

        print("")

n=1 m=10 p=107
t=   0 s=  0 sp= -1 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=   2 pswc'=  107 win'=[ 107] sf=  0 erase1=win[1=>0]=107 Sleep(8) 
t=  10 s=  1 sp=  0 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  12 pswc'=  107 win'=[ 107] sf=  1 erase1=win[2=>0]=107 Sleep(8) 
t=  20 s=  2 sp=  1 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  34 pswc'=  107 win'=[ 107] sf=  3 erase2=win[3=>0]=107 W=0 
t=  34 s=  3 sp=  2 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  74 pswc'=  107 win'=[ 107] sf=  7 erase2=win[4=>0]=107 erase2=win[5=>0]=0 erase2=win[6=>0]=0 erase2=win[7=>0]=0 W=0 
t=  74 s=  7 sp=  3 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  88 pswc'=  107 win'=[ 107] sf=  8 erase2=win[8=>0]=107 W=0 
t=  88 s=  8 sp=  7 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'= 102 pswc'=  107 win'=[ 107] sf= 10 erase2=win[9=>0]=107 erase2=win[10=>0]=0 W=0 
t= 102 s= 10 sp=  8 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'= 116 pswc'=  107 win'=

### Optimising by code de-duplication

Now that we have the working solution above, which is well commented and
describes what is going on, with somewhat verbose and repeated code, this
attempt is to simplify the algorithm while keeping the same behaviour.

In [14]:
sp = -1
s = 0
packet_sent_window_count = 0

window = [0] * n

def pwindow():
    f = "["
    for v in window:
        f += " {:3}".format(v)
    return f + "]"

reset()
print(f"n={n} m={m} p={p}")

total_sent = 0
while sim_running():
    if (s < n):
        packet_sent_expected = int(p * (s + 1) / n)
    else:
        packet_sent_expected = p

    sending = True
    while sim_running() and sending:
        dbg_time = get_time()
        packet_sent_remaining = packet_sent_expected - packet_sent_window_count
        print(f"t={dbg_time:4d} s={s:3} sp={sp:3} win={pwindow()} pse={packet_sent_expected:4} pswc={packet_sent_window_count:4} psr={packet_sent_remaining:4}", end=" ")

        # We shouldn't send more than this group at a time (especially if we're
        # sending multiple packets with one system call).
        packet_send_max = int(p * (s + 1) / n) - int(p * s / n)
        if packet_sent_remaining > packet_send_max:
            packet_send = packet_send_max
        else:
            packet_send = packet_sent_remaining
        sent = send_packets(packet_send)

        total_sent += sent
        time = get_time()
        print(f"sent={sent:4} t'={time:4d}", end=" ")

        # At the end of the loop, the current window slot must be zero. Thus all
        # cases are the same and can be simplified. Only in the case that we're
        # resending, we could increment on the existing value.
        packet_sent_window_count += sent
        window[s % n] += sent

        sf = int(time / m)
        print(f"pswc'={packet_sent_window_count:5} win'={pwindow()} sf={sf:3}", end=" ")

        sp = s
        if (sf <= s):
            if (packet_sent_expected > packet_sent_window_count):
                # If we're resending, we will add to the current slot again,
                # which means we're catching up. To smoothen out the credit
                # window, we shift back every slot if there was any window we
                # missed prior.

                # Find the next previous slot that is empty, and shift
                # everything down one slot to that one.
                #
                # We add 'n' to the index 's', so that we don't underflow.
                empty_slot = -1
                for x in range(1, n):
                    if window[(s + (n - x)) % n] == 0:
                        empty_slot = x
                        break
                if empty_slot != -1:
                    print(f"{empty_slot} {(s + (n - empty_slot)) % n}", end=" ")
                    for x in range(empty_slot, 0, -1):
                        window[(s + (n - x)) % n] = window[(s + (n - x + 1)) % n]
                        window[(s + (n - x + 1)) % n] = 0
                print("Resend!")
                continue

            s += 1

            # Given the current time and the next slot, calculate the number of
            # milliseconds to sleep
            W = s * m - time
        else:
            s = sf
            W = 0

        # Increase credits for the upcoming slot from `sp+1` up to `s` inclusive.
        for x in range(sp+1,s+1):
            print(f"erase=win[{x}=>{x%n}]={window[x%n]}", end=" ")
            packet_sent_window_count -= window[x % n]
            window[x % n] = 0

        sending = False
        if (W >= 4):
            print(F"Sleep({W})", end=" ")
            sleep_time(W)
        else:
            print(F"W={W}", end=" ")

        print("")

n=1 m=10 p=107
t=   0 s=  0 sp= -1 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=   2 pswc'=  107 win'=[ 107] sf=  0 erase=win[1=>0]=107 Sleep(8) 
t=  10 s=  1 sp=  0 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  12 pswc'=  107 win'=[ 107] sf=  1 erase=win[2=>0]=107 Sleep(8) 
t=  20 s=  2 sp=  1 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  34 pswc'=  107 win'=[ 107] sf=  3 erase=win[3=>0]=107 W=0 
t=  34 s=  3 sp=  2 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  74 pswc'=  107 win'=[ 107] sf=  7 erase=win[4=>0]=107 erase=win[5=>0]=0 erase=win[6=>0]=0 erase=win[7=>0]=0 W=0 
t=  74 s=  7 sp=  3 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'=  88 pswc'=  107 win'=[ 107] sf=  8 erase=win[8=>0]=107 W=0 
t=  88 s=  8 sp=  7 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'= 102 pswc'=  107 win'=[ 107] sf= 10 erase=win[9=>0]=107 erase=win[10=>0]=0 W=0 
t= 102 s= 10 sp=  8 win=[   0] pse= 107 pswc=   0 psr= 107 sent= 107 t'= 116 pswc'=  107 win'=[ 107] sf=

### Optimising Credit Shaping

Try to remove the window handling and deal with only how many packets we've sent
over the window.

In [15]:
reset()
print(f"n={n} m={m} p={p}")

r = 0
s = 0
packet_sent_window_count = 0
total_sent = 0
while sim_running():
    if (s < n):
        packet_sent_expected = int(p * (s + 1) / n)
    else:
        packet_sent_expected = p

    sending = True
    while sim_running() and sending:
        dbg_time = get_time()
        packet_sent_remaining = packet_sent_expected - packet_sent_window_count
        print(f"t={dbg_time:4d} s={s:3} pse={packet_sent_expected:4} pswc={packet_sent_window_count:4} psr={packet_sent_remaining:4}", end=" ")

        # We shouldn't send more than this group at a time (especially if we're
        # sending multiple packets with one system call).
        packet_send_max = int(p * (r + 1) / n) - int(p * r / n)
        if packet_sent_remaining > packet_send_max:
            packet_send = packet_send_max
        else:
            packet_send = packet_sent_remaining
        sent = send_packets(packet_send)

        total_sent += sent
        time = get_time()
        print(f"sent={sent:4} t'={time:4d}", end=" ")

        # At the end of the loop, the current window slot must be zero. Thus all
        # cases are the same and can be simplified. Only in the case that we're
        # resending, we could increment on the existing value.
        packet_sent_window_count += sent
        r += 1

        sf = int(time / m)
        print(f"pswc'={packet_sent_window_count:5} sf={sf:3}", end=" ")

        sp = s
        if (sf <= s):
            if (packet_sent_expected > packet_sent_window_count):
                print("Resend!")
                continue

            # Given the current time and the next slot, calculate the number of
            # milliseconds to sleep
            s += 1
            W = s * m - time
        else:
            s = sf
            W = 0

        # Increase credits for the upcoming slot from `sp+1` up to `s` inclusive.
        if (s >= n and packet_sent_window_count >= p):
            # Only subtract credits once we've used the time of the full window.
            if sp < n - 1:
                sp = n - 1
            slots_missed = s - sp
            credits = int(p * (r + slots_missed) / n) - int(p * r / n)
            if packet_sent_window_count > credits:
                packet_sent_window_count -= credits
            else:
                credits = packet_sent_window_count
                packet_sent_window_count = 0
            print(f"missed={slots_missed-1}; credits={credits}", end=" ")

        sending = False
        if (W >= 4):
            print(F"Sleep({W})", end=" ")
            sleep_time(W)
        else:
            print(F"W={W}", end=" ")

        print("")

n=1 m=10 p=107
t=   0 s=  0 pse= 107 pswc=   0 psr= 107 sent= 107 t'=   2 pswc'=  107 sf=  0 missed=0; credits=107 Sleep(8) 
t=  10 s=  1 pse= 107 pswc=   0 psr= 107 sent= 107 t'=  12 pswc'=  107 sf=  1 missed=0; credits=107 Sleep(8) 
t=  20 s=  2 pse= 107 pswc=   0 psr= 107 sent= 107 t'=  34 pswc'=  107 sf=  3 missed=0; credits=107 W=0 
t=  34 s=  3 pse= 107 pswc=   0 psr= 107 sent= 107 t'=  74 pswc'=  107 sf=  7 missed=3; credits=107 W=0 
t=  74 s=  7 pse= 107 pswc=   0 psr= 107 sent= 107 t'=  88 pswc'=  107 sf=  8 missed=0; credits=107 W=0 
t=  88 s=  8 pse= 107 pswc=   0 psr= 107 sent= 107 t'= 102 pswc'=  107 sf= 10 missed=1; credits=107 W=0 
t= 102 s= 10 pse= 107 pswc=   0 psr= 107 sent= 107 t'= 116 pswc'=  107 sf= 11 missed=0; credits=107 W=0 
t= 116 s= 11 pse= 107 pswc=   0 psr= 107 sent= 107 t'= 118 pswc'=  107 sf= 11 missed=0; credits=107 W=2 
t= 118 s= 12 pse= 107 pswc=   0 psr= 107 sent= 107 t'= 120 pswc'=  107 sf= 12 missed=0; credits=107 Sleep(10) 
t= 130 s= 13 pse= 107 ps